In [1]:
import torch
print('PyTorch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

PyTorch 1.9.0+cu102 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/

# import yolo project
!rm -rf yolov5
!git clone https://github.com/ultralytics/yolov5/

# import training data project
!rm -rf yolo-training-data
!git clone https://github.com/bhecquet/yolo-training-data

%cd yolov5

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content
Cloning into 'yolov5'...
remote: Enumerating objects: 8571, done.
remote: Counting objects: 100% (285/285), done.
remote: Compressing objects: 100% (175/175), done.
remote: Total 8571 (delta 163), reused 191 (delta 110), pack-reused 8286
Receiving objects: 100% (8571/8571), 9.57 MiB | 3.94 MiB/s, done.
Resolving deltas: 100% (5905/5905), done.
Cloning into 'yolo-training-data'...
remote: Enumerating objects: 9026, done.
remote: Counting objects: 100% (9026/9026), done.
remote: Compressing objects: 100% (7355/7355), done.
remote: Total 9026 (delta 1687), reused 8963 (delta 1638), pack-reused 0
Receiving objects: 100% (9026/9026), 50.07 MiB | 14.10 MiB/s, done.
Resolving deltas: 100% (1687/1687), done.
/content/yolov5


In [3]:
# création des données de test / entrainement
!python /content/yolo-training-data/generate_train_files.py /content/yolo-training-data/field-detector/dataset_extracted /content/yolo-training-data/field-detector/dataset_generated_small --output /content/yolov5/dataset

INFO:root:2721 files added to /content/yolov5/dataset/training
INFO:root:290 files added to /content/yolov5/dataset/testing


In [6]:
import torch
import os
from IPython.display import Image, clear_output 
print('PyTorch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))
best_fitness = 0.0
cfg = '/content/yolo-training-data/field-detector/data/web-generated.yaml'
#cfg = 'web-generated.yaml'
model = 'yolov5m.pt'

epoch_to_run = 2

name = os.path.basename(cfg).split(".")[0] + '-' + model.split('.')[0]


!python train.py --img 448 --batch 16 --epochs $epoch_to_run --data $cfg --weights $model --name $name --exist-ok 
weights_file_path = '/content/yolov5/runs/train/%s/weights/best.pt' % (name)
weights_file_path_google_drive = '/content/drive/My\ Drive/best_%s_%d.pt' % (name, epoch_to_run)
w = torch.load(weights_file_path)

fitness = w['best_fitness']
print("epoch completed: " + str(epoch_to_run))
print("mAp: " + str(fitness))


!cp $weights_file_path $weights_file_path_google_drive


PyTorch 1.9.0+cu102 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)
train: weights=yolov5m.pt, cfg=, data=/content/yolo-training-data/field-detector/data/web-generated.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=2, batch_size=16, imgsz=448, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache_images=False, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, entity=None, name=web-generated-yolov5m, exist_ok=True, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias=latest, local_rank=-1
github: up to date with https://github.com/ultralytics/yolov5/ ✅
YOLOv5 🚀 v5.0-327-gb60b62e torch 1.9.0+cu102 CUDA:0 (Tesla T4, 15109.75MB)

hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_b

In [ ]:
from utils import utils; utils.plot_results()

w = torch.load('weights/best.pt')
epochs_completed = str(w['epoch'])
fitness = w['best_fitness']
print("epoch completed: " + epochs_completed)
print("mAp: " + str(fitness))

In [ ]:
!python3 detect.py  --source "dataset_real/ac3bfc152e8ba5ba0b3423755fe6d234.jpg" --weights /content/drive/My\ Drive/best_web-generated_-1.pt --img-size 640 --exist-ok  --save-txt

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.25, device='', exist_ok=True, img_size=640, iou_thres=0.45, name='exp', nosave=False, project='runs/detect', save_conf=False, save_txt=True, source='dataset_real/ac3bfc152e8ba5ba0b3423755fe6d234.jpg', update=False, view_img=False, weights=['/content/drive/My Drive/best_web-generated_-1.pt'])
YOLOv3 🚀 bh1-8-g0ae89779 torch 1.8.1+cu101 CUDA:0 (Tesla P100-PCIE-16GB, 16280.875MB)

Fusing layers... 
Model Summary: 269 layers, 62589598 parameters, 0 gradients, 155.9 GFLOPS
image 1/1 /content/yolov3/dataset_real/ac3bfc152e8ba5ba0b3423755fe6d234.jpg: 384x640 8 fields, 2 radios, 3 buttons, 2 radio_with_labels, Done. (0.026s)
Results saved to runs/detect/exp
10 labels saved to runs/detect/exp/labels
Done. (0.066s)
